# Sreality - Scraping
## Jirka Zelenka
### 12.3.-24.4.2020
### Celý projekt = Scraping + Cleaning & Dropping + Vizualizace + All_In_One + PowerBI
-----------------------------------------------------------------------------

### Prerekvizity:
* chromedriver.exe v přísluěné verzi k prohlížeči
* nainstalované package
* přístup k souboru "Adresy.xlsx"
* kvalitní připojení k webu

### Obsah:
* 1) Importování Packagů
* 2) Scraping URLs
* 3) JSON - souřadnice, popis, cena
* 4) Získávání zbylých informací
* 5) Mapping adres - předešlé inzeráty + Nominatim GeoPy
* 6) Spuštění
* 7) BONUS - hudební vsuvka

-----------------------
# 1) Importování Packagů
-----------------------

In [1]:
#  1 = Scraping ###############################################################################################################################

##### Obecné ############
import pandas as pd                     # for dataframes' manipulation
from pandas import DataFrame            # for creating dataframes
import numpy as np                      # for arrays
import matplotlib as plt                # for plotting
from matplotlib.pyplot import figure    # for saving and changing size of plots

from collections import Counter         # for counting elements 
from datetime import datetime           #for actual date
import re                               # !!! relativní Novinka - regular expressions
from time import sleep                  # for sleeping (slowing down) inside a function
import random                           # for random number (sleeping)
import math                             # Round float
import time                             # Time measuring
import itertools                        # for unlisting nested lists


##### Scraping ############
import requests                         # for robots check
from bs4 import BeautifulSoup           # for parsing
from selenium import webdriver          # for browsers control
import json                             # for Requests

##### GeoPy ############        
from geopy.geocoders import Nominatim   # Geolocator   # pip install geopy  
from geopy.exc import GeocoderTimedOut  # for Error handling

##########################
# Zaítm nepoužito:
##### Widgets ############
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display
import os
from IPython.display import Image

##### Bonus - Hudba ############
import winsound                        # for Beep-sounds

##### Vizualizace ############
import seaborn as sns                  #for cool plots



import sys                             # ???

-----------------------
# 2) Scraping URLs
-----------------------

In [ ]:
print(requests.get("https://www.sreality.cz/robots.txt").text)

In [2]:
# 1 = Scraping ################################################################################################################################

def get_soup_elements(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):  
    
    browser = webdriver.Chrome()
    
    ##########################################
    # 1. Volba Prodej/Pronájem, Byty/Domy,                 --Aukce/Bez Aukce (jen pro Prodeje) zatím nechávám být, cpe se mi to doprostřed url
    ##########################################   
    
    url_x = r"https://www.sreality.cz/hledani"             
    url = url_x + "/" +  typ_obchodu + "/" +  typ_stavby

    ##########################################
    # 2. načtení webu
    ##########################################
    
    browser.get(url)    # (url).text ??
    sleep(random.uniform(1.0, 1.5))
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,'lxml') # "parser" ??
    
    elements = []    
    
    for link in soup.findAll('a', attrs={'href': re.compile("^/detail/")}):       # !!!!!!!!!!!!!!!!! změněno, protože H2 neobsahovalo všechny věci, jen nadpisek.
        link = link.get('href')   
        elements.append(link)     
    elements = elements[0::2]   

    ##########################################
    # 3. zjištění počtu listů - mělo by být optional, ale nevadí
    ##########################################
    records = soup.find_all(class_ ='numero ng-binding')[1].text
    records = re.split(r'\D', str(records))                         
    records = ",".join(records).replace(",", "")
    records = int(records)
    max_page = math.ceil(records / 20)   
    print("----------------")
    print("Scrapuji: " + str(typ_obchodu) + " " + str(typ_stavby))
    print("Celkem inzerátů: " + str(records))
    print("Celkem stránek: " + str(max_page))
    
    ##########################################
    # 4. nastavení počtu stránek  -mělo být víc promakané
    ##########################################
    if pages == 999:      # (NE)Speciální případ, chci všechny inzeráty - standardně tu bude asi kolem 600 stránek max, takže 999 je volné k použití
        pages = max_page
    
    print("Scrapuji (pouze) " + str(pages) + " stran.")
    print("----------------")
    
    ##########################################
    # 5. Scrapping zbylých listů - naštěstí v jednom okně
    ##########################################    
    
    for i in range(pages-1):   
        i = i+2
        
        sys.stdout.write('\r'+ "Strana " + str(i-1) + " = " + str(round(100*(i-1)/(pages), 2)) + "% progress. Zbývá cca: " + str(round(random.uniform(3.4, 3.8)*(pages-(i-1)), 2 )) + " sekund.")    # Asi upravím čas, na rychlejším kabelu v obýváku je to občas i tak 3 sec :O

        url2 = url + "?strana=" + str(i)
        browser.get(url2)

        sleep(random.uniform(1.0, 1.5))

        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup2 = BeautifulSoup(innerHTML,'lxml') 
        
        elements2 = []
        
        for link in soup2.findAll('a', attrs={'href': re.compile("^/detail/prodej/")}):  
            link = link.get('href') 
            elements2.append(link)  
   
        elements2 = elements2[0::2]  
        
        elements = elements + elements2     # tyto se už můžou posčítat, naštěstí, řpedtím než z nich budeme dělat elems = prvky třeba jména

    
    browser.quit()   
    
    return elements

In [3]:
#  2 = Získání URLS ###############################################################################################################################

def elements_and_ids(x):
    
    elements = pd.DataFrame({"url":x})

    def get_id(x):
        x = x.split("/")[-1]
        return x

    elements["url_id"] = elements["url"].apply(get_id)
    
    len1 = len(elements)
    #Přidáno nově, v tuto chvíli odmažu duplikáty a jsem v pohodě a šetřím si čas dál.
    elements = elements.drop_duplicates(subset = [ "url", "url_id"], keep = "first", inplace = False)   
    len2 = len(elements)                                                                             
                                                                                                      
    print("-- Vymazáno " + str(len1-len2) + " záznamů kvůli duplikaci.")
    return elements


-----------------------
# 3) JSON - souřadnice, popis, cena
-----------------------

In [4]:
# 3 = získání Souřadnic, Ceny a Popisu = z JSON ################################################################################################################################  

def get_coords_price_meters(x):
    
    url = "https://www.sreality.cz/api/cs/v2/estates/" + str(x)
    
    response = requests.get(url)
    byt = json.loads(response.text, encoding = 'UTF-8')                                
    try:
        coords = (byt["map"]["lat"], byt["map"]["lon"])
    except:
        coords = (0.01, 0.01)  #Pro případ neexisutjících souřadnic
    try:
        price = byt["price_czk"]["value_raw"] 
    except:
        price = -1
        
    try:
        description = byt["meta_description"]
    except:
        description = -1
    
    return coords, price, description


# Severní Šířka = latitude
# výchoDní / zápaDní Délka = longitude

def latitude(x):                   #Rozdělí souřadnice na LAT a LON
    x = str(x).split()[0][1:8]
    return x

def longitude(x):
    x = str(x).split()[1][0:7]
    return x


-----------------------
# 4) Získávání zbylých informací
-----------------------

In [5]:
#  4 = Prodej + Dům + Pokoje = z URL ###############################################################################################################################

def characteristics(x):
    x = x.split("/")
    buy_rent = x[2]
    home_house = x[3]
    rooms = x[4]
    
    return buy_rent, home_house, rooms

#  5 =  Plocha z Popisu ###############################################################################################################################

# Upraveno pro čísla větší než 1000 aby je to vzalo
# Zároveň se to vyhne velikost "Dispozice", "Atpyický", atd.

def plocha(x):
    try:
        metry = re.search(r'\s[12]\s\d{3}\s[m]', x)[0] # SPecificky popsáno: Začíná to mezerou, pak 1 nebo 2, pak mezera, pak tři čísla, mezera a pak "m"
        metry = metry.split()[0] + metry.split()[1]     # Separuju Jedničku + stovky metrů, bez "m"
    except:
        try:
            metry = re.search(r'\s\d{2,3}\s[m]', x)[0]  #Mezera, pak 1-3 čísla, mezera a metr
            metry = metry.split()[0]                    # Separuju čísla, bez "m"
        except:
            metry = -1
    return metry

-----------------------
# 5) Mapping adres - předešlé inzeráty + Nominatim GeoPy
-----------------------

### 1) Státní správa
    https://www.statnisprava.cz/rstsp/ciselniky.nsf/i/CZ0201
    - problémy s abecedou, Brno-město nemá obce atd
### 2) Wiki
    https://cs.wikipedia.org/wiki/Seznam_katastr%C3%A1ln%C3%ADch_%C3%BAzem%C3%AD_v_okrese_Bene%C5%A1ov
    - taky celý dost na houby, nezačal jsem
### 3) Volby.cz
    https://www.volby.cz/pls/kv2018/kv31?xjazyk=CZ&xid=1
    - Bylo by hezké, jsou to tables, easy to scrapp, jen jiný počet než v 1) (75 vs 77 okresů)
### 4) Staťák - excel
    https://www.czso.cz/csu/czso/pocet-obyvatel-v-obcich-za0wri436p?fbclid=IwAR1haSspuynZB8Awn08WDriMkUcsUCz4fHH9Pw2CwMDVGHPGJERxaqbrVg8
    - Tohle asi bude top, Zuzka mě zachránila


* 14 krajů
* 77 okresů podle státní správy včetně PRAHA
* 6258 obcí a újezdů  K 27. květnu 2016 (ČSÚ)  


In [6]:
# 6 = Adresy z předešlých inzerátů a short_coords ###############################################################################################################################

# Vytvoření ořezaných souřadnic, přesnost je dostatečná, lépe se najdou duplikáty
def short_coords(x):
    """
    x = x.astype(str)   # Bylo potřeba udělat string - ale Tuple se blbě převádí - vyřešil jsem uložením a načtením skrz excel
    """
    
    x1 = re.split(r'\W+', x)[1] + "."+re.split(r'\W+', x)[2]
    x1 = round(float(x1), 4)

    x2 = re.split(r'\W+', x)[3] + "."+re.split(r'\W+', x)[4]
    x2 = round(float(x2), 4)

    return (x1, x2)

#############################

# Napmapuje až 80 % Adres z předešlých inzerátů
def adress_old(x):  

    adresy = pd.read_excel("Adresy.xlsx")
    adresy = adresy[["oblast", "město", "okres", "kraj", "url_id", "short_coords"]]
    
    #Nejlepší napárování je toto:
    # alternativně Inner a Left minus řádky s NaNs a funguje stejně)
    
    x.short_coords = x.short_coords.astype(str)                              # získat string na souřadnice, protože v Načteném adresáři je mám už taky jako string
    data = pd.merge(x, adresy, on=["short_coords", "url_id"], how = "left")  #upraveno matchování na url_ID + short_coords, je to tak iv Adresáří, je to jednoznačné, jsou tam unikátní. 
                                                                            # Pokud si v dalším kroku dostáhnu ke starému url_id a k nové coords ještě novou adresu, tak pak se mi uloží do Adresáře nová kombiance ID + short_coord a je to OK
                                                                             # Viz funkce"update_databáze_adres() kde je totéž info
            
    print("-- Počet doplněných řádků je: " + str(len(data[~data.kraj.isna()])) + ", počet chybějících řádků je: "   + str(len(data[data.kraj.isna()])))
    
    return data

In [7]:
# 7 = Adresy - zbývající přes GeoLocator ###############################################################################################################################

def adress_new(x):

# Pozn. - je to random, závislost rychlosti na user_agent, i na format_string se nepovedlo potvrdit - ale dokumentace user-agent uvíádí jako povinnost
# Timeout na 20s  zrušil Errory - None záleží na verzi geopy !! viz dokumentace
#Rychlost a úspěch velmi záleží na připojení. Ryhlost 0.2s - 10s na záznam.
# Problém s Too many requests se "spraví přes noc", kdyžtak - nebo viz stackoverflow - nastavit user-agent (https://stackoverflow.com/questions/22786068/how-to-avoid-http-error-429-too-many-requests-python)
  
    geolocator = Nominatim(timeout = 20, user_agent = "JZ_Sreality")   # Pomohlo změnit jméno, proti "Error 403" !!        
    location = geolocator.reverse(x.strip("())"))   
                                                    # Reverse samotné znamená obrácené vyhleádvání = souřadnice -> Adresa
    try:
        oblast  = location[0].split(",")[-7]
    except:
        oblast  = -1
    try:
        město = location[0].split(",")[-6]
    except:
        město  = -1
    try:
        okres = location[0].split(",")[-5]
    except:
        okres  = -1
    try:
        kraj = location[0].split(",")[-4]  
    except:
        kraj  = -1       
    
    time.sleep(0.5)
    return oblast, město, okres, kraj

##################################################################

# Pomocná funkce, opakuje předchozí funkci pořád dokola dokud neprojde bez Erroru
def repeat_adress(x):
    try:
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))
    except GeocoderTimedOut:
        print("Another try")
        x["oblast"], x["město"],  x["okres"] ,  x["kraj"]  = zip(*x['coords'].map(adress_new))


# 8 = Merging adres ###############################################################################################################################
# Aplikuje předchozí funkci pouze na řídky, které ještě nemají doplněné adresy z kroku 6.)

def adress_merging(x):

    data_new = x.copy()          
    bool_series = pd.isnull(data_new.kraj)                                   
    data_new = data_new[bool_series]     #subset s chybějícími adresami   
        
    repeat_adress(data_new)
        
    data_all = pd.concat([x, data_new], join_axes=[x.columns])   
    data_all = data_all[~pd.isnull(data_all.kraj)]
    data_all = data_all.sort_index()

    data = data_all.copy()
    
    return data


-----------------------
# 6) Spuštění
-----------------------

In [8]:

# Parametry:
# "prodej"/ "pronájem"
# "byty"/"domy"
# pages = 1- X, případně 999 = Všechny strany !

def scrap_all(typ_obchodu = "prodej", typ_stavby = "byty", pages = 1):
    
    # Scrapni data - hezky komunikuje = cca 50 min
    data = get_soup_elements(typ_obchodu = typ_obchodu, typ_stavby = typ_stavby, pages = pages)
    print( "1/8 Data scrapnuta, získávám URLs.")
    
    # 2 = Získání URLS
    data = elements_and_ids(data)
    data.to_excel(r"a1_URLs_prodej_byty.xlsx")
    print( "2/8 Získány URL, nyní získávám Souřadnice, Ceny a Popis - několik minut...")
    
    
    # 3 = získání Souřadnic, Ceny a Popisu = z JSON
    data["coords"], data["cena"], data["popis"] = zip(*data['url_id'].map(get_coords_price_meters))
    data["lat"] = data["coords"].apply(latitude)
    data["lon"] = data["coords"].apply(longitude)
    data.to_excel(r"a2_Souřadnice_prodej_byty.xlsx")
    print( "3/8 Získány Souřadnice, Ceny a Popis, nyní získávám informace z URLs.")
   
    # 4 = Prodej + Dům + Pokoje = z URL
    data["prodej"], data["dům"],  data["pokoje"] = zip(*data['url'].map(characteristics))
    print("4/8 Získány informace z URLs, nyní získávám informace z popisu.")
    
    # 5 =  Plocha z Popisu
    data["plocha"] = data['popis'].apply(plocha)
    data.to_excel(r"a3_Popisky_prodej_byty.xlsx")
    print( "5/8 Získány informace z Popisu, nyní mapuji Adresy z předešlých inzerátů.")
    
   
    # 6 = Adresy z předešlých inzerátů a short_coords
    data = pd.read_excel(r"a3_Popisky_prodej_byty.xlsx")   # Abych se vyhnul konverzi TUPLE na STRING, což není triviální, tak si to radši uložím a znova načtu a získám stringy rovnou. Snad mi to nerozbije zbytek
    data["short_coords"] = data["coords"].apply(short_coords)
    data_upd = adress_old(data)                                # Tady nepotřebuji maping, protože se nesnažím něco nahodit na všechny řádky, ale merguju celé datasety
    data = data_upd.copy()
    print( "6/8 Namapovány Adresy z předešlých inzerátů, nyní stahuji nové Adresy - několik minut...")            # Přidat do printu počet řádků, kolik mám a kolik zbývá v 7. kroku

    # 7-8 = Adresy - zbývající přes GeoLocator + Merging

    try:                                    # !!! Riskuju že zas něco selže, jako USER- AGENT posledně...
        data_upd = adress_merging(data)    #Přidáno TRY pro situace, kdy už mám všechyn adresy z OLD a nejde nic namapovat !
        data = data_upd.copy()
        data.to_excel(r"a4_SCRAPED_prodej_byty.xlsx")
        print("7+8/8 Získány nové adresy + mergnuto dohromady. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 1.")
    
    except:
        data.to_excel(r"a4_SCRAPED_prodej_byty.xlsx")
        print("7+8/8 ŽÁDNÉ nové adresy. Celková délka datasetu: "+ str(len(data)) + ". Konec Fáze 1.")
    

    return data

In [9]:
data = scrap_all(pages=2)

----------------
Scrapuji: prodej byty
Celkem inzerátů: 14050
Celkem stránek: 703
Scrapuji (pouze) 2 stran.
----------------
Strana 1 = 50.0% progress. Zbývá cca: 3.5 sekund.1/8 Data scrapnuta, získávám URLs.
-- Vymazáno 0 záznamů kvůli duplikaci.
2/8 Získány URL, nyní získávám Souřadnice, Ceny a Popis - několik minut...
3/8 Získány Souřadnice, Ceny a Popis, nyní získávám informace z URLs.
4/8 Získány informace z URLs, nyní získávám informace z popisu.
5/8 Získány informace z Popisu, nyní mapuji Adresy z předešlých inzerátů.
-- Počet doplněných řádků je: 22, počet chybějících řádků je: 18
6/8 Namapovány Adresy z předešlých inzerátů, nyní stahuji nové Adresy - několik minut...
7+8/8 Získány nové adresy + mergnuto dohromady. Celková délka datasetu: 40. Konec Fáze 1.


### BONUS - Hudební vsuvka aplikovatelná do scrapingu jako alarm

In [ ]:
import winsound
import time 

sec = 300  # milliseconds - for Beep
half_sec = 150
pause = 1   # Seconds - for Sleep

# záhadně to nebere násobky floatem ani dělení integerem i když ot ve výsledku je celé číslo

# Hz
A3 = 220
C4 = 262   #261.63
D4 = 294   #293.67
E4 = 330   #329.63
F4 = 350   #349.23
G4 = 392
A4 = 440
C5 = 523   #523.25

koef = 2

def we_are_the_champions():
     
    winsound.Beep(koef*F4, 4*sec)
    winsound.Beep(koef*E4, sec)
    winsound.Beep(koef*F4, sec)
    winsound.Beep(koef*E4, 3*sec)
    winsound.Beep(koef*C4, 2*sec)
    winsound.Beep(koef*A3, 3*half_sec)
    winsound.Beep(koef*D4, 3*half_sec)
    winsound.Beep(koef*A3, 10*sec)
    time.sleep(pause)
    winsound.Beep(koef*C4, sec)
    winsound.Beep(koef*F4, 4*sec)
    winsound.Beep(koef*G4, 2*sec)
    winsound.Beep(koef*A4, 2*sec)
    winsound.Beep(koef*C5, 4*sec)
    winsound.Beep(koef*A4, 2*sec)
    winsound.Beep(koef*D4, sec)
    winsound.Beep(koef*E4, sec)
    winsound.Beep(koef*D4, 6*sec)

    
    
we_are_the_champions()